In [ ]:
# -*- coding: utf-8 -*-
"""Pressure Drop Calculator — with editable ε (mm)"""

from IPython.display import display, HTML
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import pi
import ipywidgets as W
from IPython.display import display, clear_output, HTML as IHTML

# =======================
# LIGHT THEME (Dashboard style)
# =======================
THEME = {
    "page_bg":   "#F5F7FB",
    "card_bg":   "#FFFFFF",
    "border":    "#E6EAF1",
    "text":      "#0F172A",
    "muted":     "#64748B",
    "primary":   "#4F46E5",
    "accent":    "#14B8A6",
    "good":      "#10B981",
    "grid":      "#E2E8F0"
}

# ---------- CSS (Light dashboard) ----------
display(HTML(f"""
<style>
body, .jp-Notebook, .jp-Cell, .jp-RenderedHTMLCommon {{
  background: {THEME["page_bg"]} !important;
  color: {THEME["text"]} !important;
  font-family: Inter, ui-sans-serif, system-ui, -apple-system, Segoe UI, Roboto, Arial, "Noto Sans";
}}
.dashboard-card {{
  background: {THEME["card_bg"]};
  border: 1px solid {THEME["border"]};
  border-radius: 18px;
  padding: 16px 18px;
  box-shadow: 0 8px 20px rgba(15,23,42,0.06);
  margin: 10px 0;
}}
.hdr {{ font-weight: 800; font-size: 20px; margin-bottom: 6px; color:{THEME["text"]}; }}
.subhdr {{
  font-weight: 700; font-size: 14px; margin: 10px 0 6px; color:{THEME["muted"]};
  text-transform: uppercase; letter-spacing: .06em;
}}
.kpill {{
  display:inline-block; padding:6px 10px; background:{THEME["page_bg"]};
  border:1px solid {THEME["border"]}; border-radius:999px; font-size:12px; font-weight:700; color:{THEME["text"]};
}}
.widget-label {{ color:{THEME["text"]} !important; font-weight:600; }}
input, select, textarea {{
  background:#FFF !important; color:{THEME["text"]} !important;
  border:1px solid {THEME["border"]} !important; border-radius:12px !important;
}}
.widget-text input {{ padding:6px 10px; }}
.widget-dropdown select {{ padding:6px 10px; }}
.widget-button.button-primary {{ background:{THEME["primary"]} !important; color:#fff !important; font-weight:700; border-radius:12px; }}
.widget-button.button-info {{ background:{THEME["accent"]} !important; color:#fff !important; font-weight:700; border-radius:10px; }}
.widget-button.button-warning {{ background:#EF4444 !important; color:#fff !important; font-weight:700; border-radius:10px; }}
.dataframe tbody tr:nth-child(odd) {{ background-color: #FBFDFF; }}
.dataframe th, .dataframe td {{ border-color: {THEME["border"]} !important; }}

/* Keep disabled inputs readable and obvious */
input:disabled {{
  background: #F9FAFB !important;
  color: #374151 !important;
  opacity: 1 !important;
}}
</style>
"""))

# =======================
# Document (header card)
# =======================
explanation = f"""
<div class="dashboard-card">
  <div class="hdr">Pressure Drop Calculator — Documentation</div>
  <div class="subhdr">Steps / Flow of Formulas</div>
  <ol style="margin:0">
    <li>Convert volumetric flow: Q_SI = Q / 3600 [m³/s]</li>
    <li>Area: A = π·D²/4 [m²]</li>
    <li>Velocity: v = Q_SI / A [m/s]</li>
    <li>Reynolds: Re = ρ·v·D / μ [–]</li>
    <li>Relative roughness: ε/D [–] (ε entered in mm → converted to m)</li>
    <li>Friction model:
      <ul>
        <li><b>Auto</b>: if Re&lt;2300 → Laminar (f=64/Re), else Turbulent (Haaland)</li>
        <li><b>Laminar</b>: force f = 64/Re</li>
        <li><b>Turbulent</b>: force Haaland</li>
      </ul>
    </li>
    <li>Major loss: ΔP_major = f·(L/D)·(ρ·v²/2)</li>
    <li>Minor loss: ΔP_minor = K_total·(ρ·v²/2)</li>
    <li>Static head: ΔP_static = ρ·g·Δz</li>
    <li>Total: ΔP_total = ΔP_major + ΔP_minor + ΔP_static</li>
    <li>Convert: ΔP_total_bar = ΔP_total / 1e5 [bar]</li>
  </ol>
</div>
"""
display(HTML(explanation))

# =======================
# Data libraries
# =======================
MATERIALS_MM = {
    "PVC / CPVC / PE (very smooth)": 0.0015,
    "Drawn tubing / glass":          0.0010,
    "Stainless steel (commercial)":  0.015,
    "Commercial steel / wrought iron": 0.045,
    "Cast iron (new)":               0.26,
    "Galvanized iron":               0.15,
    "Concrete (smooth)":             0.30,
    "Concrete (rough)":              3.00,
    "Custom (enter ε mm)":           None,     # <-- choose this to type your own ε
}

K_LIBRARY = {
    "90° Elbow (R/D ≈ 1)": 0.90,
    "45° Elbow": 0.40,
    "Gate Valve (open)": 0.15,
    "Globe Valve (open)": 10.0,
    "Swing Check Valve": 2.0,
    "Sudden Expansion": 1.0,
    "Sudden Contraction": 0.5,
    "Tee (through run)": 0.6,
    "Tee (through branch)": 1.8,
    "Entrance (sharp-edged)": 0.5,
    "Exit": 1.0,
    "Strainer (clean)": 2.0,
    "Custom": 0.0,
}

# =======================
# Core calculations
# =======================
def friction_factor(Re, rel_rough, mode="auto"):
    Re = float(Re)
    if Re <= 0 or np.isnan(Re):
        return np.nan

    def f_lam(r): return 64.0 / r

    def f_haaland(r, rr):
        return (1.0 / (-1.8 * np.log10((rr/3.7)**1.11 + 6.9/r)))**2

    if mode == "laminar":   return f_lam(Re)
    if mode == "turbulent": return f_haaland(Re, rel_rough)
    return f_lam(Re) if Re < 2300 else f_haaland(Re, rel_rough)

def calc_dp_for_pipe(D, Q_m3h, L, rho, mu, eps_m, K_total, dz, mode="auto"):
    Q = Q_m3h / 3600.0
    A = pi * D**2 / 4.0
    v = Q / A
    Re = rho * v * D / mu
    rr = (eps_m if eps_m is not None else 0.0) / D if D > 0 else 0.0
    f = float(friction_factor(Re, rr, mode))
    dp_major  = f * (L / D) * (rho * v**2 / 2.0)
    dp_minor  = K_total * (rho * v**2 / 2.0)
    dp_static = rho * 9.80665 * dz
    dp_total  = dp_major + dp_minor + dp_static

    if mode == "laminar":
        regime = "Laminar (forced 64/Re)"
    elif mode == "turbulent":
        regime = "Turbulent (forced Haaland)"
    else:
        regime = "Laminar (auto)" if Re < 2300 else "Turbulent (auto, Haaland)"

    return {
        "D_m": D,
        "Velocity_m_s": v,
        "Re": Re,
        "Regime/f": regime,
        "f_Darcy": f,
        "dP_major_bar": dp_major/1e5,
        "dP_minor_bar": dp_minor/1e5,
        "dP_static_bar": dp_static/1e5,
        "dP_total_bar": dp_total/1e5,
    }

# =======================
# Widget helpers
# =======================
def num_field(desc, val, step, width=220):
    return W.FloatText(value=val, step=step, description=desc,
                       style={'description_width':'120px'},
                       layout=W.Layout(width=f"{width}px"))

def dd_field(desc, options, value=None, width=420):
    return W.Dropdown(options=options, value=(value if value is not None else options[0]),
                      description=desc, style={'description_width':'120px'},
                      layout=W.Layout(width=f"{width}px"))

# =======================
# Pipe panels (editable names, start empty)
# =======================
def make_pipe_panel(default_id):
    chk   = W.Checkbox(value=False, description='', indent=False,
                       layout=W.Layout(width="28px"))
    name  = W.Text(value='', placeholder='Pipe name',
                   layout=W.Layout(width="160px"))

    idm   = num_field("ID (m):", default_id, 0.001, 180)

    mat   = dd_field("Material:", list(MATERIALS_MM.keys()),
                     "PVC / CPVC / PE (very smooth)", 340)

    # Wider ε input; locked unless "Custom"
    default_eps = MATERIALS_MM["PVC / CPVC / PE (very smooth)"]
    epsmm = num_field("ε (mm):", default_eps, 0.0001, 220)

    # lock/unlock based on material
    epsmm.disabled = MATERIALS_MM[mat.value] is not None
    epsmm.placeholder = "type roughness in mm (e.g., 0.045)" if MATERIALS_MM[mat.value] is None else ""

    def on_mat_change(change):
        val = MATERIALS_MM[change["new"]]
        if val is None:
            epsmm.disabled = False
            epsmm.placeholder = "type roughness in mm (e.g., 0.045)"
        else:
            epsmm.value = val
            epsmm.disabled = True
            epsmm.placeholder = ""

    mat.observe(on_mat_change, names="value")

    row = W.HBox([chk, name, idm, mat, epsmm])
    return {"box": row, "chk": chk, "name": name, "id": idm, "mat": mat, "epsmm": epsmm}

pipe1 = make_pipe_panel(0.150)
pipe2 = make_pipe_panel(0.200)
pipe3 = make_pipe_panel(0.250)

left_col = W.VBox([
    W.HTML('<div class="subhdr">Select pipes</div>'),
    pipe1["box"], pipe2["box"], pipe3["box"]
])

# =======================
# Constants + friction model
# =======================
Q_m3h = num_field("Flow Q (m³/h):", 823.0, 1.0, 240)
L_m   = num_field("Length L (m):", 10.0, 0.1, 240)
dz_m  = num_field("Elevation Δz (m):", 0.0, 0.1, 240)
rho   = num_field("ρ (kg/m³):", 998.0, 1.0, 240)
mu    = num_field("μ (Pa·s):", 0.001, 0.0001, 240)
friction_model = dd_field("Friction model:",
    [("Auto (pick by Re)", "auto"), ("Laminar (64/Re)", "laminar"), ("Turbulent (Haaland)", "turbulent")],
    "auto", 240
)
const_box = W.VBox([W.HTML('<div class="subhdr">Constants</div>'), Q_m3h, L_m, dz_m, rho, mu, friction_model])

# =======================
# K-factor builder (bigger inputs)
# =======================
k_elem = dd_field("Element:", list(K_LIBRARY.keys()),
                  "90° Elbow (R/D ≈ 1)", width=380)

k_val  = num_field("K value:", K_LIBRARY[k_elem.value], 0.05, width=220)  # wider
k_qty  = W.IntText(value=0, description="Qty:",
                   style={'description_width':'60px'},
                   layout=W.Layout(width="140px"))  # wider

k_add  = W.Button(description="Add",  button_style="info")
k_clr  = W.Button(description="Clear", button_style="warning")

k_table_out = W.Output()
k_total_out = W.HTML('<span class="kpill">Total K: 0.000</span>')
K_rows = []

def refresh_k_table():
    if K_rows:
        df = pd.DataFrame(K_rows)
        df["subtotal"] = df["K"] * df["qty"]
        with k_table_out:
            clear_output(); display(df)
        k_total = float(df["subtotal"].sum())
    else:
        with k_table_out:
            clear_output(); display(IHTML("<i>No elements added.</i>"))
        k_total = 0.0
    k_total_out.value = f'<span class="kpill">Total K: {k_total:.3f}</span>'
    return k_total

def on_elem_change(change):
    k_val.value = K_LIBRARY[k_elem.value]
k_elem.observe(on_elem_change, names="value")

def on_add_clicked(b):
    if k_qty.value > 0:
        K_rows.append({"name": k_elem.value, "K": float(k_val.value), "qty": int(k_qty.value)})
        k_qty.value = 0
        refresh_k_table()

def on_clr_clicked(b):
    K_rows.clear()
    refresh_k_table()

k_add.on_click(on_add_clicked)
k_clr.on_click(on_clr_clicked)

k_input_row = W.HBox([k_elem, k_val, k_qty], layout=W.Layout(width='100%'))
k_box = W.VBox([
    W.HTML('<div class="subhdr">K-factors</div>'),
    k_input_row,
    W.HBox([k_add, k_clr]),
    k_table_out,
    k_total_out
])

# =======================
# Run & Output
# =======================
run_btn = W.Button(description="Run comparison", button_style="primary")
out = W.Output()
ui = W.HBox([left_col, W.VBox([const_box, k_box, run_btn])])
display(ui, out)

def bar_with_labels(index, values, title, ylabel):
    plt.figure(figsize=(7,4))
    bars = plt.bar(index, values, color=[THEME["primary"], THEME["accent"], THEME["good"]][:len(index)])
    plt.title(title, color=THEME["text"])
    plt.ylabel(ylabel, color=THEME["text"])
    plt.grid(True, axis="y", color=THEME["grid"])
    for b in bars:
        plt.text(b.get_x()+b.get_width()/2, b.get_height(), f"{b.get_height():.3f}", ha='center', va='bottom')
    plt.tight_layout()
    plt.show()

@out.capture(clear_output=True)
def on_run_clicked(b):
    selected = []
    for panel in (pipe1, pipe2, pipe3):
        if panel["chk"].value:
            eps_mm = panel["epsmm"].value
            selected.append({
                "name": (panel["name"].value.strip() or f"Pipe {len(selected)+1}"),
                "D": float(panel["id"].value),
                "eps_m": float(eps_mm)/1000.0 if eps_mm is not None else 0.0
            })

    if not selected:
        print("Please select at least one pipe.")
        return

    Q   = float(Q_m3h.value)
    L   = float(L_m.value)
    dz  = float(dz_m.value)
    rho_v = float(rho.value)
    mu_v  = float(mu.value)
    mode  = friction_model.value
    K_total = refresh_k_table()

    print(f"Using Total K = {K_total:.3f}  |  Friction model = {mode}")

    rows = []
    for p in selected:
        res = calc_dp_for_pipe(p["D"], Q, L, rho_v, mu_v, p["eps_m"], K_total, dz, mode=mode)
        res["Name"] = p["name"]
        rows.append(res)

    df = pd.DataFrame(rows).set_index("Name")
    cols = ["D_m","Velocity_m_s","Re","Regime/f","f_Darcy","dP_major_bar","dP_minor_bar","dP_static_bar","dP_total_bar"]
    display(df[cols].round(5))

    bar_with_labels(df.index.tolist(), df["dP_total_bar"].values, f"Total ΔP (bar) — L={L} m, Q={Q} m³/h", "ΔP_total (bar)")

    df.to_csv("pressure_drop_results.csv")
    print("Saved: pressure_drop_results.csv")

run_btn.on_click(on_run_clicked)


Output()